In [ ]:
##all the import neccesary to carry out the computation
import numpy as np
import urllib2
import matplotlib.pyplot as plt
from sklearn.cross_validation import KFold
from sklearn.preprocessing import PolynomialFeatures
from numpy.linalg import inv,solve,pinv
from sklearn import linear_model
from math import exp,fabs
import time
%matplotlib inline

## fucnction ot convert X nto Z matrix
##take X and d-degree of polynomail as input and returns Z matrix
def define_Z(X,d):
    poly = PolynomialFeatures(d)
    Z = poly.fit_transform(X)
    return Z
    
    
##function to train for polynomial function
##input :- Z and Y labels 
##returns :- computed theta
def single_poly_train(Z,Y):
    
    #print Z
    pin = pinv(Z)
    #print pin.shape
    Q = np.dot(pin,Y)
    return Q
    
    
    
    
## function to predict lables
## takes new feature vector and theta as input an return preducted value of Y
def single_poly_predict(Z,Q):
    return np.transpose(np.dot(np.transpose(Q),np.transpose(Z)))
    
    
    
##function to train linear model 
##input :- feature vector and Label
##returns :- value of theta
def single_linear_train(X,Y):
    m = len(Y)
    
    sum_x = sum(X[:,0])
    
    sum_x_2 = 0
    for i in X[:,0]:
        sum_x_2 += i**2
    
    sum_y = sum(Y[:,0])
    
    sum_x_y = 0
    
    for i in zip(X[:,0],Y[:,0]):
        sum_x_y += i[0]*i[1]
    
    
    A = np.array([[m,sum_x],[sum_x,sum_x_2]])
    B = np.array([[sum_y],[sum_x_y]])

    Q = solve(A, B)
    return Q


##Function to predict lables by linear model
## input :- new features and theta
##returns predicted value of Y
def single_linear_predict(X,Q):
    Y_hat = []
    for i in range(len(X[:,0])):
        Y_hat.append(Q[0,0] + X[i,0]*Q[1,0])
    return np.array(Y_hat)
    
##function to compute mean square error
def training_error(Y_hat,Y):
    return np.mean((Y_hat - Y) ** 2)
    
        
#function to compute testing error using ten croos fold for single variable data set
def ten_cross_fold(X,Y):
    accuracies = []
    cv = KFold(len(Y),10)
    for train_id,test_id in cv:
        
        Q = single_linear_train(X[train_id],Y[train_id])
        
        Y_hat = single_linear_predict(X[test_id],Q)
        
        
        error = training_error(Y_hat,Y[test_id])
        
        accuracies.append(error)
    
    return np.mean(accuracies)

#function to compute testing error using ten croos fold for multiple variable data set
def ten_cross_fold_poly(Z,Y):
    accuracies = []
    cv = KFold(len(Y),10)
    for train_id,test_id in cv:
        
        Q = single_poly_train(Z[train_id],Y[train_id])
        
        Y_hat = single_poly_predict(Z[test_id],Q)
        
        
        error = training_error(Y_hat,Y[test_id])
        
        accuracies.append(error)
    
    return np.mean(accuracies)    





In [ ]:

##single feature and linear model
##calls all the apropriete function to get the training and testing error
def single_linear_model(X,Y):
    
    train_error = []
    test_error = []
    
    Q = single_linear_train(X,Y)            ##trains and computes theta
    Y_hat = single_linear_predict(X,Q)      ##predicts Y_hat
    train_error.append(training_error(Y_hat,Y))
    #print train_error
    test_error.append(ten_cross_fold(X,Y))
    #print test_error
    return (train_error,test_error,Y_hat)
    



In [ ]:


##single feature and non linear model
##calls all the apropriete function to get the training and testing error
def single_ploy_model(X,Y):
    
    train_error = []
    test_error = []

    for i in range(2,16):

        Z = define_Z(X,i)
        #print Z.shape
        Q = single_poly_train(Z,Y)
        #print Q.shape
        Y_hat = single_poly_predict(Z,Q)
        #print Y_hat.shape
        #print Y.shape
        train_error.append(training_error(Y_hat,Y))
        
        test_error.append(ten_cross_fold_poly(Z,Y))
    return (train_error,test_error)



In [ ]:
## iterative solution to predict theta using the learning rate and 
##then use it to predict Y and compute the testing error

##Input :- feature vector, Y, threshold, learning rate and degree of the polynomial of define Z


def iterative_solution(X,Y,thres,lr,d):    
    best = False
    Z = define_Z(X,d)
    theta = np.zeros(shape=(Z.shape[1],1))
    theta.fill(0.5)
    max_it = 0
    
    
    while best==False:
        theta_pre = theta
        grad = 2 * np.dot(Z.transpose(),(np.dot(Z,theta_pre) - Y))
        theta =  theta_pre - (lr*grad)
        J_theta_pre = np.dot(np.transpose(np.dot(Z,theta_pre) - Y),(np.dot(Z,theta_pre)-Y))
        J_theta = np.dot(np.transpose(np.dot(Z,theta) - Y),(np.dot(Z,theta)-Y))
        max_it += 1
        #print J_theta
        
        if fabs((J_theta[0] - J_theta_pre[0])[0]) < thres or max_it == 100: ##condition to check to stop the while loop
            best = True
            #print theta
    print "ERROR:", np.mean((np.dot(Z,theta) - Y)**2) ##error from the iteration method
            

In [ ]:
##main function to do all the functinality of the single feature
##Take feature and Y as input

def single_feature(X,Y):
    if (len(X[1]) == 1 ):
        plt.plot(X,Y,'ro')               ##plotting the data to gain some idea of the complexity
        plt.xlabel("X")
        plt.ylabel("Y")
        plt.title("svar-set3")
        plt.show()


        ##fitting my linear model and computing traning and testing error
        train_linear,test_linear,Y_hat = single_linear_model(X,Y)
        #print len(Y_hat)
        #print len(Y)



        ##plotting tthe model fitted by me
        plt.plot(X,Y,'ro',X,Y_hat,lw = 4)
        plt.xlabel("X")
        plt.ylabel("Y")
        plt.title("Model fit by me for svar-set3")

        plt.show()

        print "training error by me for linear regression:",train_linear
        print "testing error by me for linear regression:",test_linear

        ##fitting the ready made python function for linear regression
        regr = linear_model.LinearRegression()
        regr.fit(X, Y)
        train_linear_python = np.mean(((regr.predict(X) - Y)/(Y)) ** 2)


        ##using ten cross fold validation to get the testing error for ready made linear regression model
        accuracies = []
        cv = KFold(len(Y),10)

        for train_id,test_id in cv:    
            regr.fit(X[train_id],Y[train_id])
            error = np.mean(((regr.predict(X[test_id]) - Y[test_id])/(Y[test_id])) ** 2)
            accuracies.append(error)
        test_linear_python = np.mean(accuracies)

        ##plotting the model by scikit learn

        plt.plot(X,Y,'ro',X,regr.predict(X),lw = 4)
        plt.xlabel("X")
        plt.ylabel("Y")
        plt.title("ready made linear regression for svar-set3")

        plt.show()

        print "training error by readymade model for linear regression:%s \n" % train_linear_python
        print "testing error by readymade model for linear regression:%s \n" % test_linear_python
       

        train_poly,test_poly= single_ploy_model(X,Y)

        print "training error by me for  non linear regression(1,15): %s \n"  % train_poly
        print "testing error by me for non linear regression(1,15): %s \n" % test_poly
        index = test_poly.index(min(test_poly)) + 2

        #train = (a_train + b_train)
        #test =  (a_test + b_test)
        #print train
        #print test

        #plt.boxplot(train)
        #plt.show()
        
        ##plotting the change in the training error with the polynomial
        plt.plot(range(2,16),train_poly,'-bo')
        plt.xlabel("degree")
        plt.ylabel("training error")
        plt.title("svar-set2")
        plt.show()

        
        ##plotting the change in the testing error with the polynomial
        plt.plot(range(2,16),test_poly,'-bo')
        plt.xlabel("degree")
        plt.ylabel("test error")
        plt.title("svar-set1")
        plt.show()


        Z = define_Z(X,index)
        Q = single_poly_train(Z,Y)
        Y_hat = single_poly_predict(Z,Q)


        ##model fit with the polynomial degree with the least error
        plt.plot(X,Y,'ro',X,Y_hat,'bo',lw = 2)
        plt.xlabel("X")
        plt.ylabel("Y")
        plt.title("Model fit by me for polynomial %d for svar-set1" % index)
        plt.show()
 

In [ ]:
##function to carry out all the opeartion on the multi feature data set


def multi_feature(X,Y):  
    test_error = []
    for i in range (1,4):  ## fitting the model of polynomail degree 1 to 3
        Z = define_Z(X,i)
        Q = single_poly_train(Z,Y)
        #print Q
        Y_hat = single_poly_predict(Z,Q)
        test_error.append(ten_cross_fold_poly(Z,Y))
    print test_error
    min_error = test_error.index(min(test_error))+1
    print "minimum error with degree: ",min_error
    
    ## plotting the testing error to compare each degree and find the best degree 
    plt.plot(range(1,4),test_error,'-bo')
    plt.xlabel("degree")
    plt.ylabel("error")
    plt.title("Model fit by me for multifeature polynomial")
    plt.show() 
    return min_error

In [ ]:
##guassian function to compute the similarity between two vectors
##input :- two vectors x and y, value of sigma

def gaussian_similarity(X,Y,sigma = 3.0):
    m = np.dot(np.transpose(X-Y),(X-Y))
    return exp(m*(-1/(2*(sigma**2)))) 


In [ ]:
## calculating the gram matrix for a given matrix X
##input :- feature matrix X
##returns the gram matrix

def gram_matrix(X):
    m = len(X)
    gram = np.zeros(shape=(m,m))
    for i in range(m):
        x = X[i]
        for j in range(m):        
            y = X[j]
            gram[i][j] += gaussian_kernel(x,y)
    return gram
    

In [ ]:
## function to print the error produces by the kernel method
## Takes feature vector and labels as input

def kernel_solution(X,Y):
    
    gram = gram_matrix(X) ##computes the gram matrix
    
    alpha = np.dot(inv(gram),Y) ##computes the parameter alpha
    
    y_hat =  np.zeros(shape=(m,1)) ##initializes the predictor matrix


    for i in range(m):

        x = X[i]
        z = np.zeros(shape=(m,1))  ##Z matrix containing the similarity 

        for j in range(m):

            z[j][0] += gaussian_similarity(X[j],x)
            
        y_hat[i][0] = np.dot(np.transpose(alpha),z)
        
    print "Error by kermel method: ",np.mean((y_hat - Y) ** 2) ##prints the mean square error


In [ ]:

### FOR SYNTHETIC DATA SET####


data = urllib2.urlopen("http://www.cs.iit.edu/~agam/cs584/data/regression/svar-set3.dat")
data = data.read()
data = data.split('\n')
data = data[:len(data)-1]
X = []
Y=[]

##clean the data set and prepare the feature vector
for i in data[5:]:
    i = i.split()
    a = []
    for j in i[:-1]:
        a.append(float(j))
    X.append(a)
    Y.append(float(i[len(i)-1]))


X = np.array(X)
Y = np.reshape(np.array(Y),(len(Y),1))

##condition to check if the dataset is single variate##

if (len(X[1])==1):
    single_feature(X,Y) ##call to single_feature function to carry out all the operation on single variate dataset
    print "------------------------AFTER REDUCING THE AMOUNT OF DATA-----------------------------"
    m=int(.10*(len(X))) ##redusing  the amount of data set
    single_feature(X[0:m],Y[0:m])
    
##if the data set id multi variate##
else:
    ##time function to calculate the time take by the function
    time_func = time.time
    startTime = time_func()
    min_error_at = multi_feature(X,Y) ##call to multi_feature function to carry out all the operation on multi variate dataset
    endTime = time_func()
    print min_error_at
    print "Time Taken:",endTime-startTime
    time_dim.append(endTime-startTime)
    
    print "---------------------------WITH ITERATIVE SOLUTION----------------------------------"
    
    ## to check the testing error for the iterative approach using gradient desent
    time_func = time.time
    startTime = time_func()
    iterative_solution(X,Y,0.01,0.0000001,min_error_at) ##calling the function with different values of LR to find
                                                        ##the optimal LR for a given data set
    
    endTime = time_func()
    print "Time Taken:",endTime-startTime
    
    print "----------------------------KERNEL SOLUTION-----------------------------------------"
    
    time_func = time.time
    startTime = time_func()
    kernel_solution(X,Y) ## calling the kernel function 
    endTime = time_func()
    print "Time Taken:",endTime-startTime
    

    

In [ ]:
###REAL DATA SET WITH 5 FEATURES AND ONE PREDICTED VALUE ###

## same code as above but with different data set and cleaning method
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00291/airfoil_self_noise.dat"
data = urllib2.urlopen(url)
data = data.read()
data = data.split('\r')
data = data[:-1]
X = []
Y=[]
for i in data[:len(data)]:
    i = i.split('\t')
    a = []
    for j in i[:-1]:
        a.append(float(j))
    X.append(a)
    Y.append(float(i[len(i)-1]))
X = np.array(X)
Y = np.reshape(np.array(Y),(len(Y),1))


time_func = time.time
startTime = time_func()
min_error_at = multi_feature(X,Y)
endTime = time_func()
print min_error_at
print "Time Taken:",endTime-startTime
time_dim.append(endTime-startTime)
    
print "---------------------------WITH ITERATIVE SOLUTION----------------------------------"
time_func = time.time
startTime = time_func()
iterative_solution(X,Y,0.01,0.000001,min_error_at)
endTime = time_func()
print "Time Taken:",endTime-startTime